# Results combination and analysis

In [1]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [2]:
%reload_kedro
import csv
import glob
import io
import logging
import os
import subprocess
from datetime import datetime

import plotly.express as px
import plotly.io as pio
import requests

pio.renderers.default = "iframe"

import re

import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import yaml
from GPSOdyssey import Kepler
from IPython.core.interactiveshell import InteractiveShell

# Ipython configs
from IPython.display import HTML, display
from shapely.geometry import LineString

display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

2022-06-03 13:08:35,245 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-03 13:08:35,314 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-06-03 13:08:35,315 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-06-03 13:08:35,325 - root - INFO - Registered line magic `run_viz`


## Find partition keys

In [3]:
df_producer_geo_all = catalog.load("syn_pop_scenarios_local_20220601")
waste_all = catalog.load("waste_gen_scenarios_local_20220601")
results_all = catalog.load("resource_summary_table_20220601_decentral")
keys = list(df_producer_geo_all)
keys = [x for x in keys if x[0] != "."]

2022-06-03 13:08:35,512 - kedro.io.data_catalog - INFO - Loading data from `syn_pop_scenarios_local_20220601` (PartitionedDataSet)...
2022-06-03 13:08:35,527 - kedro.io.data_catalog - INFO - Loading data from `waste_gen_scenarios_local_20220601` (PartitionedDataSet)...
2022-06-03 13:08:35,538 - kedro.io.data_catalog - INFO - Loading data from `resource_summary_table_20220601_decentral` (PartitionedDataSet)...


## Extract and summaries

In [4]:
i = 0

In [5]:
all_results = []
for key in keys:
    results = results_all[key]()
    results = results.loc[results["Unnamed: 0"] == "Total"]
    results["scenario"] = key
    all_results.append(results)
all_results = pd.concat(all_results)
all_results = all_results.drop(columns=["Unnamed: 0", "Collection day"])
all_results[
    ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
] = all_results[
    ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
].fillna(
    0
)

## Addapt travel and offload service time

In [6]:
all_results.head()

,Vehicle,Route,Offloads,Bins collected,Units served,Demand collected (ton),Time collecting (h),Time travelling (h),Time at treatment facility (h),Route duration (h),Traveling distance (km),Collecting distance (km),Total route distance (km),Electrical consumption (kWh),Tons disposed at @ Offload 2,Tons disposed at @ Offload 3,scenario
18,3,18,52,38603,38603,432.33,78.31,13.44,91.00,182.69,601.78,419.25,1021.03,2209.14,172.04,260.30,households_001
18,3,18,52,38655,38655,432.51,78.39,12.99,91.00,182.32,581.80,419.04,1000.84,2185.00,144.57,287.96,households_002
18,3,18,51,38714,38714,433.33,78.50,13.79,89.25,181.46,617.07,419.27,1036.34,2229.01,156.33,277.02,households_003
18,3,18,51,38693,38693,433.33,78.46,13.29,89.25,180.91,594.15,419.02,1013.17,2201.02,134.88,298.45,households_004
18,3,18,52,38674,38674,435.89,78.43,13.48,91.00,182.82,602.38,419.21,1021.59,2215.10,128.20,307.69,households_005


In [7]:
offload_travel = all_results["Offloads"] * ( 90 / 60 ) # hours
offload_distance = offload_travel * 45 # (at 45 km / h)
all_results["Time at treatment facility (h)"] = all_results["Time at treatment facility (h)"] - offload_travel
all_results['Time travelling (h)'] = all_results[ 'Time travelling (h)'] + offload_travel
all_results['Traveling distance (km)'] = all_results['Traveling distance (km)'] + offload_distance
all_results['Total route distance (km)'] = all_results['Total route distance (km)'] + offload_distance
all_results["Offloads per route"] = all_results["Offloads"] / all_results["Route"]
all_results["Time per route (h)"] = all_results["Route duration (h)"] / all_results["Route"]
all_results["Time travelling per route (h)"] = all_results["Time travelling (h)"] / all_results["Route"]
all_results["Time collecting per route (h)"] = all_results["Time collecting (h)"] / all_results["Route"]

In [8]:
all_results.head()

,Vehicle,Route,Offloads,Bins collected,Units served,Demand collected (ton),Time collecting (h),Time travelling (h),Time at treatment facility (h),Route duration (h),Traveling distance (km),Collecting distance (km),Total route distance (km),Electrical consumption (kWh),Tons disposed at @ Offload 2,Tons disposed at @ Offload 3,scenario,Offloads per route,Time per route (h),Time travelling per route (h),Time collecting per route (h)
18,3,18,52,38603,38603,432.33,78.31,91.44,13.00,182.69,4111.78,419.25,4531.03,2209.14,172.04,260.30,households_001,2.888889,10.149444,5.080000,4.350556
18,3,18,52,38655,38655,432.51,78.39,90.99,13.00,182.32,4091.80,419.04,4510.84,2185.00,144.57,287.96,households_002,2.888889,10.128889,5.055000,4.355000
18,3,18,51,38714,38714,433.33,78.50,90.29,12.75,181.46,4059.57,419.27,4478.84,2229.01,156.33,277.02,households_003,2.833333,10.081111,5.016111,4.361111
18,3,18,51,38693,38693,433.33,78.46,89.79,12.75,180.91,4036.65,419.02,4455.67,2201.02,134.88,298.45,households_004,2.833333,10.050556,4.988333,4.358889
18,3,18,52,38674,38674,435.89,78.43,91.48,13.00,182.82,4112.38,419.21,4531.59,2215.10,128.20,307.69,households_005,2.888889,10.156667,5.082222,4.357222


## Store results

In [9]:
%reload_kedro
catalog.save("resource_scenario_results_20220601_decentral", all_results)

2022-06-03 13:08:36,103 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-03 13:08:36,171 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-06-03 13:08:36,172 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-06-03 13:08:36,181 - root - INFO - Registered line magic `run_viz`
2022-06-03 13:08:36,182 - kedro.io.data_catalog - INFO - Saving data to `resource_scenario_results_20220601_decentral` (CSVDataSet)...


## Early graphs

In [10]:
fig = px.histogram(all_results, x="Demand collected (ton)")
fig

fig = px.histogram(all_results, x="Units served")
fig

In [11]:
fig = px.histogram(all_results, x="Route duration (h)")
fig

In [12]:
all_results.columns

Index(['Vehicle', 'Route', 'Offloads', 'Bins collected', 'Units served',
       'Demand collected (ton)', 'Time collecting (h)', 'Time travelling (h)',
       'Time at treatment facility (h)', 'Route duration (h)',
       'Traveling distance (km)', 'Collecting distance (km)',
       'Total route distance (km)', 'Electrical consumption (kWh)',
       'Tons disposed at @ Offload 2', 'Tons disposed at @ Offload 3',
       'scenario', 'Offloads per route', 'Time per route (h)',
       'Time travelling per route (h)', 'Time collecting per route (h)'],
      dtype='object')

In [13]:
fig = px.histogram(all_results, x="Route")
fig

fig = px.histogram(all_results, x="Offloads")
fig

In [14]:
fig = px.histogram(all_results, x="Total route distance (km)")
fig

In [24]:
fig = px.scatter(all_results, x="Units served", y="Route duration (h)")
fig
fig = px.scatter(all_results, x="Units served", y="Total route distance (km)")
fig

In [26]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Route duration (h)")
fig
fig = px.scatter(all_results, x="Demand collected (ton)", y="Total route distance (km)")
fig

In [17]:
fig = px.scatter(all_results, x="Units served", y="Demand collected (ton)")
fig

In [18]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Total route distance (km)", color="Units served")
fig

In [19]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Route duration (h)", color="Units served")
fig

In [20]:
all_results.loc[all_results["Route duration (h)"] >= 182]

,Vehicle,Route,Offloads,Bins collected,Units served,Demand collected (ton),Time collecting (h),Time travelling (h),Time at treatment facility (h),Route duration (h),Traveling distance (km),Collecting distance (km),Total route distance (km),Electrical consumption (kWh),Tons disposed at @ Offload 2,Tons disposed at @ Offload 3,scenario,Offloads per route,Time per route (h),Time travelling per route (h),Time collecting per route (h)
18,3,18,52,38603,38603,432.33,78.31,91.44,13.0,182.69,4111.78,419.25,4531.03,2209.14,172.04,260.30,households_001,2.888889,10.149444,5.080000,4.350556
18,3,18,52,38655,38655,432.51,78.39,90.99,13.0,182.32,4091.80,419.04,4510.84,2185.00,144.57,287.96,households_002,2.888889,10.128889,5.055000,4.355000
18,3,18,52,38674,38674,435.89,78.43,91.48,13.0,182.82,4112.38,419.21,4531.59,2215.10,128.20,307.69,households_005,2.888889,10.156667,5.082222,4.357222
18,3,18,52,38685,38685,434.72,78.44,91.41,13.0,182.78,4110.05,419.08,4529.13,2210.30,175.37,259.36,households_010,2.888889,10.154444,5.078333,4.357778
18,3,18,52,38735,38735,436.35,78.54,91.58,13.0,183.03,4117.24,419.45,4536.70,2222.12,179.38,256.99,households_011,2.888889,10.168333,5.087778,4.363333
18,3,18,52,38641,38641,435.22,78.39,91.37,13.0,182.67,4107.67,419.54,4527.22,2209.13,159.37,275.86,households_016,2.888889,10.148333,5.076111,4.355000
18,3,18,52,38605,38605,432.86,78.30,91.21,13.0,182.44,4101.32,418.78,4520.10,2196.43,167.13,265.72,households_018,2.888889,10.135556,5.067222,4.350000
18,3,18,52,38605,38605,432.70,78.27,91.49,13.0,182.66,4112.55,417.71,4530.26,2207.54,156.46,276.27,households_019,2.888889,10.147778,5.082778,4.348333
18,3,18,52,38668,38668,436.11,78.42,91.39,13.0,182.71,4107.92,419.19,4527.11,2210.04,160.26,275.84,households_020,2.888889,10.150556,5.077222,4.356667
18,3,18,52,38671,38671,436.79,78.34,91.53,13.0,182.78,4115.15,416.52,4531.67,2214.40,139.97,296.84,households_022,2.888889,10.154444,5.085000,4.352222


In [21]:
from pandas_profiling import ProfileReport

2022-06-03 13:08:41,762 - visions.backends - INFO - Pandas backend loaded 1.4.2
2022-06-03 13:08:41,768 - visions.backends - INFO - Numpy backend loaded 1.22.3
2022-06-03 13:08:41,769 - visions.backends - INFO - Pyspark backend NOT loaded
2022-06-03 13:08:41,770 - visions.backends - INFO - Python backend loaded


In [22]:
profile = ProfileReport(all_results, title="Waste generation scenario analysis - 20220601 - decentral")

In [23]:
profile.to_file("../data/08_reporting/Waste generation scenario analysis - 20220601 - decentral.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/missingno/missingno.py:72: MatplotlibDeprecationWarning:

The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.



Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]